In [14]:
%load_ext kedro.ipython

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [15]:
%reload_ext kedro.ipython

[05/16/24 20:05:54] INFO     Registered line magic '%reload_kedro'                                   ]8;id=126140;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=417749;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#51\51]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=14166;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=919121;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#53\53]8;;\

                    INFO     Resolved project path as: /Users/gavinlou/Developer/gasai.             ]8;id=723498;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=455378;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#164\164]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Kedro project gasAI                                                    ]8;id=377879;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=831571;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#134\134]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=363076;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=908803;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#135\135]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=381049;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=95806;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#141\141]8;;\

In [16]:
df = catalog.load("s3_conc_aligned_df")

                    INFO     Loading data from s3_conc_aligned_df (ParquetDataset)...           ]8;id=740055;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=526110;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#483\483]8;;\

In [17]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

class GroupDataset(Dataset):
    def __init__(self, X, y, groups):
        self.X = X.unsqueeze(2)
        self.y = y.unsqueeze(1)
        self.groups = groups
        self.group_to_indices = self._group_indices()

    def _group_indices(self):
        group_to_indices = {}
        for idx, group in enumerate(self.groups):
            if group not in group_to_indices:
                group_to_indices[group] = []
            group_to_indices[group].append(idx)
        return group_to_indices

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.groups[idx]

class ExperimentSampler:
    def __init__(self, group_to_indices, batch_size):
        self.group_to_indices = group_to_indices
        self.batch_size = batch_size
        self.group_order = list(group_to_indices.keys())
        np.random.shuffle(self.group_order)  # Shuffle the order of groups

    def __iter__(self):
        for group in self.group_order:
            indices = self.group_to_indices[group]
            for i in range(0, len(indices), self.batch_size):
                yield indices[i:i + self.batch_size]

    def __len__(self):
        total_batches = 0
        for indices in self.group_to_indices.values():
            total_batches += (len(indices) + self.batch_size - 1) // self.batch_size
        return total_batches

# Define the features and target
features = ['timestamp_bin', 'A1_Resistance']
target = 'resistance_ratio'
X_tensor = df[features]
y_tensor = df[target]
groups = df['exp_no']
batch_size = 32

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_tensor.values, dtype=torch.float32)
y_tensor = torch.tensor(y_tensor.values, dtype=torch.float32)
# Convert groups to NumPy array
groups = np.array(groups)

# Create dataset
dataset = GroupDataset(X_tensor, y_tensor, groups)

# Split the data into training and testing sets using GroupShuffleSplit
gss = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
train_idx, test_idx = next(gss.split(X_tensor, y_tensor, groups=groups))

train_dataset = GroupDataset(X_tensor[train_idx], y_tensor[train_idx], groups[train_idx])
test_dataset = GroupDataset(X_tensor[test_idx], y_tensor[test_idx], groups[test_idx])

# Create experiment samplers
train_sampler = ExperimentSampler(train_dataset.group_to_indices, batch_size)
test_sampler = ExperimentSampler(test_dataset.group_to_indices, batch_size)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_sampler=train_sampler)
test_loader = DataLoader(test_dataset, batch_sampler=test_sampler)

# # Example: iterating through the train data loader
# print("Train Batches:")
# for batch_indices in train_loader:
#     # Flatten the list of indices
#     batch_indices = [item for sublist in batch_indices for item in sublist]
#     X_batch = train_dataset.X[batch_indices]
#     y_batch = train_dataset.y[batch_indices]
#     print(X_batch.shape, y_batch.shape, batch_indices)



In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("mps" if torch.cuda.is_available() else "cpu")

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        out, _ = self.lstm(x, (h0,c0))  
        out = out[:, -1, :]
        out = self.fc(out)
        return out




In [19]:
from torch.optim import Adam
from math import sqrt

input_size = X_tensor.shape[1]
hidden_size = 32
num_layers = 2
num_classes = 1

device = torch.device("mps" if torch.cuda.is_available() else "cpu")

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

def calculate_rmse(outputs, labels):
    mse = torch.mean((outputs - labels) ** 2)  # Calculate MSE as a tensor
    return torch.sqrt(mse)  # Return RMSE as a tensor



def train_model(model, train_loader, val_loader, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels, _ in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            # print(inputs.shape)
            inputs = inputs.transpose(1, 2)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation
        model.eval()

        # During validation
        val_rmse = 0.0
        total_samples = 0
        with torch.no_grad():
            for inputs, labels, _ in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                inputs = inputs.transpose(1, 2)
                outputs = model(inputs)
                batch_rmse = calculate_rmse(outputs, labels) * inputs.size(0)
                val_rmse += batch_rmse.item()  # Properly use .item() to convert tensor to float
                total_samples += inputs.size(0)

        val_rmse /= total_samples
        print(f'Epoch {epoch+1} Train Loss: {epoch_loss:.4f} Val RMSE: {val_rmse:.4f}')

# Training the model
train_model(model, train_loader, test_loader, n_epochs=15)


Epoch 1 Train Loss: 0.1696 Val RMSE: 0.4124
Epoch 2 Train Loss: 0.1798 Val RMSE: 0.4157
Epoch 3 Train Loss: 0.1852 Val RMSE: 0.4170
Epoch 4 Train Loss: 0.1863 Val RMSE: 0.4171
Epoch 5 Train Loss: 0.1890 Val RMSE: 0.4110
Epoch 6 Train Loss: 0.1890 Val RMSE: 0.4170
Epoch 7 Train Loss: 0.1886 Val RMSE: 0.4162
Epoch 8 Train Loss: 0.1880 Val RMSE: 0.4166
Epoch 9 Train Loss: 0.1886 Val RMSE: 0.4108
Epoch 10 Train Loss: 0.1893 Val RMSE: 0.4125
Epoch 11 Train Loss: 0.1881 Val RMSE: 0.4162
Epoch 12 Train Loss: 0.1896 Val RMSE: 0.4114
Epoch 13 Train Loss: 0.1892 Val RMSE: 0.4106
Epoch 14 Train Loss: 0.1896 Val RMSE: 0.4102
Epoch 15 Train Loss: 0.1894 Val RMSE: 0.4105


In [22]:
# i want to examine the content of the loader to see if timestamps are being passed correctly

for inputs, label, group in train_loader:
    print(inputs)
    break


tensor([[[0.0000e+00],
         [9.4908e+05]],

        [[1.0000e+00],
         [9.4923e+05]],

        [[2.0000e+00],
         [9.4867e+05]],

        [[3.0000e+00],
         [9.4892e+05]],

        [[4.0000e+00],
         [9.4617e+05]],

        [[5.0000e+00],
         [9.4409e+05]],

        [[6.0000e+00],
         [9.3979e+05]],

        [[7.0000e+00],
         [9.3708e+05]],

        [[8.0000e+00],
         [9.3304e+05]],

        [[9.0000e+00],
         [9.2785e+05]],

        [[1.0000e+01],
         [9.2466e+05]],

        [[1.1000e+01],
         [9.1862e+05]],

        [[1.2000e+01],
         [9.1457e+05]],

        [[1.3000e+01],
         [9.1084e+05]],

        [[1.4000e+01],
         [9.0572e+05]],

        [[1.5000e+01],
         [9.0388e+05]],

        [[1.6000e+01],
         [8.9849e+05]],

        [[1.7000e+01],
         [8.9548e+05]],

        [[1.8000e+01],
         [8.9100e+05]],

        [[1.9000e+01],
         [8.8762e+05]],

        [[2.0000e+01],
         [8.8570e

In [21]:
print(X_tensor.shape)

torch.Size([469995, 2])
